# Dlib Model

##### Dlib is an open source framework used to figure out facial landmarks which are the salient features of faces like eyes ,  mouth and eyebrows. But here we going to use this to identy or localize face in an image

In [1]:
import dlib
import cv2


In [2]:
face_detector = dlib.get_frontal_face_detector()

In [3]:
%%time

img = cv2.imread("images/single_face.jpg")
bbox = face_detector(img)
print("Number of Faces ==>",len(bbox))

for faces in bbox:
    x,y,width,height = faces.left(),faces.top(),faces.right() - faces.left(),faces.bottom()-faces.top()
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),2)

cv2.imwrite("output_images/dlib_single_face.jpg",img)

Number of Faces ==> 1
CPU times: user 114 ms, sys: 0 ns, total: 114 ms
Wall time: 113 ms


In [4]:
%%time

img = cv2.imread("images/multiple_faces.jpg")
bbox = face_detector(img)

print("Number of Faces ==>",len(bbox))

for faces in bbox:
    x,y,width,height = faces.left(),faces.top(),faces.right() - faces.left(),faces.bottom()-faces.top()
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),2)
    
cv2.imwrite("output_images/dlib_multiple_faces.jpg",img)

Number of Faces ==> 11
CPU times: user 384 ms, sys: 12 ms, total: 396 ms
Wall time: 401 ms


In [5]:
type(bbox)

dlib.rectangles

In [6]:
%%time

img = cv2.imread("images/group_faces.jpg")
bbox = face_detector(img)

print("Number of Faces ==>",len(bbox))

for faces in bbox:
    x,y,width,height = faces.left(),faces.top(),faces.right() - faces.left(),faces.bottom()-faces.top()
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),5)
    
cv2.imwrite("output_images/dlib_group_faces.jpg",img)

Number of Faces ==> 39
CPU times: user 1.94 s, sys: 56.7 ms, total: 1.99 s
Wall time: 2.01 s


# MTCNN Model

In [7]:
from mtcnn import MTCNN
import cv2

Using TensorFlow backend.


In [8]:
face_detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
%%time
img = cv2.cvtColor(cv2.imread("images/single_face.jpg"), cv2.COLOR_BGR2RGB)
face_info = face_detector.detect_faces(img)

print("Number of Faces ==>",len(face_info))

for faces in face_info:
    x,y,width,height = faces["box"]
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),2)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

cv2.imwrite("output_images/mtcnn_single_face.jpg",img)

Number of Faces ==> 1
CPU times: user 708 ms, sys: 53.4 ms, total: 762 ms
Wall time: 461 ms


In [10]:
face_info

[{'box': [208, 122, 155, 207],
  'confidence': 1.0,
  'keypoints': {'left_eye': (243, 205),
   'mouth_left': (249, 276),
   'mouth_right': (311, 274),
   'nose': (271, 244),
   'right_eye': (306, 203)}}]

In [11]:
%%time
img = cv2.cvtColor(cv2.imread("images/multiple_faces.jpg"), cv2.COLOR_BGR2RGB)
face_info = face_detector.detect_faces(img)

print("Number of Faces ==>",len(face_info))

for faces in face_info:
    x,y,width,height = faces["box"]
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),2)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

cv2.imwrite("output_images/mtcnn_multiple_face.jpg",img)

Number of Faces ==> 12
CPU times: user 4.06 s, sys: 308 ms, total: 4.37 s
Wall time: 2.22 s


In [12]:
%%time
img = cv2.cvtColor(cv2.imread("images/group_faces.jpg"), cv2.COLOR_BGR2RGB)
face_info = face_detector.detect_faces(img)

print("Number of Faces ==>",len(face_info))

for faces in face_info:
    x,y,width,height = faces["box"]
    cv2.rectangle(img,(x,y),(x+width,y+height),(255,0,0),5)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

cv2.imwrite("output_images/mtcnn_group_face.jpg",img)

Number of Faces ==> 42
CPU times: user 20.4 s, sys: 2.58 s, total: 23 s
Wall time: 11.3 s


# Tiny Face Tensorflow

In [13]:
import tensorflow as tf
import tiny_face_model
import util
import cv2
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import pylab as pl
import time
import os
import sys
from scipy.special import expit
import glob

MAX_INPUT_DIM = 5000.0

In [14]:
def _calc_scales():
        raw_h, raw_w = raw_img.shape[0], raw_img.shape[1]
        min_scale = min(np.floor(np.log2(np.max(clusters_w[normal_idx] / raw_w))),
                        np.floor(np.log2(np.max(clusters_h[normal_idx] / raw_h))))
        max_scale = min(1.0, -np.log2(max(raw_h, raw_w) / MAX_INPUT_DIM))
        scales_down = pl.frange(min_scale, 0, 1.)
        scales_up = pl.frange(0.5, max_scale, 0.5)
        scales_pow = np.hstack((scales_down, scales_up))
        scales = np.power(2.0, scales_pow)
        return scales
    
def _calc_bounding_boxes():
    # threshold for detection
    _, fy, fx, fc = np.where(prob_cls_tf > prob_thresh)

    # interpret heatmap into bounding boxes
    cy = fy * 8 - 1
    cx = fx * 8 - 1
    ch = clusters[fc, 3] - clusters[fc, 1] + 1
    cw = clusters[fc, 2] - clusters[fc, 0] + 1

    # extract bounding box refinement
    Nt = clusters.shape[0]
    tx = score_reg_tf[0, :, :, 0:Nt]
    ty = score_reg_tf[0, :, :, Nt:2*Nt]
    tw = score_reg_tf[0, :, :, 2*Nt:3*Nt]
    th = score_reg_tf[0, :, :, 3*Nt:4*Nt]

    # refine bounding boxes
    dcx = cw * tx[fy, fx, fc]
    dcy = ch * ty[fy, fx, fc]
    rcx = cx + dcx
    rcy = cy + dcy
    rcw = cw * np.exp(tw[fy, fx, fc])
    rch = ch * np.exp(th[fy, fx, fc])

    scores = score_cls_tf[0, fy, fx, fc]
    tmp_bboxes = np.vstack((rcx - rcw / 2, rcy - rch / 2, rcx + rcw / 2, rcy + rch / 2))
    tmp_bboxes = np.vstack((tmp_bboxes / s, scores))
    tmp_bboxes = tmp_bboxes.transpose()
    return tmp_bboxes


def overlay_bounding_boxes(raw_img, refined_bboxes, lw):
    """Overlay bounding boxes of face on images.
    Args:
      raw_img:
        A target image.
      refined_bboxes:
        Bounding boxes of detected faces.
      lw: 
        Line width of bounding boxes. If zero specified,
        this is determined based on confidence of each detection.
    Returns:
      None.
    """

    # Overlay bounding boxes on an image with the color based on the confidence.
    for r in refined_bboxes:
        _score = expit(r[4])
        cm_idx = int(np.ceil(_score * 255))
        rect_color = [int(np.ceil(x * 255)) for x in util.cm_data[cm_idx]]  # parula
        _lw = lw
        if lw == 0:  # line width of each bounding box is adaptively determined.
            bw, bh = r[2] - r[0] + 1, r[3] - r[0] + 1
            _lw = 1 if min(bw, bh) <= 20 else max(2, min(3, min(bh / 20, bw / 20)))
            _lw = int(np.ceil(_lw * _score))

        _r = [int(x) for x in r[:4]]
        cv2.rectangle(raw_img, (_r[0], _r[1]), (_r[2], _r[3]), rect_color, _lw)

In [15]:
weight_file_path = "model.pkl"
prob_thresh = 0.5
nms_thresh = 0.1
lw = 3
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, [1, None, None, 3]) # n, h, w, c

    # Create the tiny face model which weights are loaded from a pretrained model.
    model = tiny_face_model.Model(weight_file_path)
    score_final = model.tiny_face(x)

    # Find image files in data_dir.
    filenames = []
    for ext in ('*.png', '*.gif', '*.jpg', '*.jpeg',"*.JPG"):
        filenames.extend(glob.glob(os.path.join("images/", ext)))
#     print(filenames)

    # Load an average image and clusters(reference boxes of templates).
    with open(weight_file_path, "rb") as f:
        _, mat_params_dict = pickle.load(f)

    average_image = model.get_data_by_key("average_image")
    clusters = model.get_data_by_key("clusters")
    clusters_h = clusters[:, 3] - clusters[:, 1] + 1
    clusters_w = clusters[:, 2] - clusters[:, 0] + 1
    normal_idx = np.where(clusters[:, 4] == 1)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for filename in filenames:
            fname = filename.split(os.sep)[-1]
            raw_img = cv2.imread(filename)
            raw_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)
            raw_img_f = raw_img.astype(np.float32)

      

            scales = _calc_scales()
            start = time.time()

            # initialize output
            bboxes = np.empty(shape=(0, 5))

            # process input at different scales
            for s in scales:
#                 print("Processing {} at scale {:.4f}".format(fname, s))
                img = cv2.resize(raw_img_f, (0, 0), fx=s, fy=s, interpolation=cv2.INTER_LINEAR)
                img = img - average_image
                img = img[np.newaxis, :]

                # we don't run every template on every scale ids of templates to ignore
                tids = list(range(4, 12)) + ([] if s <= 1.0 else list(range(18, 25)))
                ignoredTids = list(set(range(0, clusters.shape[0])) - set(tids))

                # run through the net
                score_final_tf = sess.run(score_final, feed_dict={x: img})

                # collect scores
                score_cls_tf, score_reg_tf = score_final_tf[:, :, :, :25], score_final_tf[:, :, :, 25:125]
                prob_cls_tf = expit(score_cls_tf)
                prob_cls_tf[0, :, :, ignoredTids] = 0.0



                tmp_bboxes = _calc_bounding_boxes()
                bboxes = np.vstack((bboxes, tmp_bboxes)) # <class 'tuple'>: (5265, 5)


            print("time {:.2f} secs for {}".format(time.time() - start, fname))

            # non maximum suppression
            # refind_idx = util.nms(bboxes, nms_thresh)
            refind_idx = tf.image.non_max_suppression(tf.convert_to_tensor(bboxes[:, :4], dtype=tf.float32),
                                                       tf.convert_to_tensor(bboxes[:, 4], dtype=tf.float32),
                                                       max_output_size=bboxes.shape[0], iou_threshold=nms_thresh)
            refind_idx = sess.run(refind_idx)
            refined_bboxes = bboxes[refind_idx]
            print("Number of Faces ==>",len(refined_bboxes))

            overlay_bounding_boxes(raw_img, refined_bboxes, lw)
            raw_img = cv2.cvtColor(raw_img, cv2.COLOR_RGB2BGR)
            cv2.imwrite("output_images/tiny_tensor_{}.jpg".format(filename.split("/")[1]), raw_img)



/home/whirldata/tensorflow-dev/lib/python3.5/site-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: numpy.arange
  
/home/whirldata/tensorflow-dev/lib/python3.5/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: numpy.arange
  import sys


time 19.69 secs for single_face.jpg
Number of Faces ==> 1
time 80.45 secs for multiple_faces.jpg
Number of Faces ==> 12
time 78.29 secs for group_faces.jpg
Number of Faces ==> 41


# Face Recognition Module 

In [16]:
import cv2
import face_recognition

In [17]:
%%time
img = cv2.imread("images/single_face.jpg")
face_locations = face_recognition.face_locations(img)

print("Number of Faces ==>",len(face_locations))

for top, right, bottom, left in face_locations:
        cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 255), 2)
cv2.imwrite("output_images/face_recognition_single_face.jpg",img)

Number of Faces ==> 1
CPU times: user 329 ms, sys: 3.91 ms, total: 333 ms
Wall time: 331 ms


In [18]:
%%time
img = cv2.imread("images/multiple_faces.jpg")
face_locations = face_recognition.face_locations(img)

print("Number of Faces ==>",len(face_locations))

for top, right, bottom, left in face_locations:
        cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 255), 2)
cv2.imwrite("output_images/face_recognition_multiple_face.jpg",img)

Number of Faces ==> 11
CPU times: user 1.24 s, sys: 3.27 ms, total: 1.24 s
Wall time: 1.23 s


In [19]:
%%time
img = cv2.imread("images/group_faces.jpg")
face_locations = face_recognition.face_locations(img)

print("Number of Faces ==>",len(face_locations))

for top, right, bottom, left in face_locations:
        cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 255), 5)
cv2.imwrite("output_images/face_recognition_group_face.jpg",img)

Number of Faces ==> 39
CPU times: user 7.18 s, sys: 209 ms, total: 7.39 s
Wall time: 7.36 s
